In [50]:
# IMPORTS
import pandas as pd

import tkinter as tk
from tkinter import filedialog as fd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

import re
from unidecode import unidecode

from datetime import datetime

import bcrypt

In [51]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')

# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']

# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [52]:
# funcion para ejecutar una script SQL
# estado = 0 para select
# estado = 1 para update
def dbQuery(q, estado = 0):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            if estado==1:
                conn.commit()  # Commit the transaction
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [53]:
# GENERAR CODIGO SLUG, CODIGO UNICO
def abbreviate_para_slug(name):
    # Eliminar acentos y otros caracteres especiales
    name = unidecode(name)
    # Mantener solo caracteres alfanuméricos, espacios y guiones
    name = re.sub(r"[^\w\s-]", "", name)
    # Convertir a minúsculas
    name = name.lower()
    # Reemplazar espacios y guiones por un solo guión
    abbreviated = re.sub(r"[\s-]+", "-", name)
    return abbreviated.strip('-')

In [54]:
# optiene la fecha actual
now = datetime.now()
formatted_now = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_now)

2023-08-31 07:27:38


In [55]:
file_name = 'G:/OneDrive - Ingenio Azucarero Guabira S.A/AGRO-CITTCA/DATA PARA CARGAR/PROPIEDADES.XLSX'

In [56]:
data = pd.read_excel(file_name)
data.head(3)

,cod_prop,nom_prop,cod_ca,nom_ca,area,cuadrante,user_id,periodo_id
0,2,2 - JOSE TREJO--SEGUNDINA IRIARTE,18354,TREJO ESCALERA JOSE,0.80,359,413,15
1,4,4 - SAN JORGE--CATALA,1843,CATALA ABREGO SEGISMUNDO,44.00,359,630,15
2,5,5 - SAN FELIX--CATALA,1858,CATALA CAMPOS SEGISMUNDO,204.36,359,593,15


In [57]:
data.shape

(1459, 8)

In [58]:
# script para insertar registro de prop
INSERT INTO propiedad 
(codigo, nombre, slug, superficie_total, foto, estado, cuadrante, usuario_id, periodo_id, created_at, updated_at)
VALUES 
('2', '2 - JOSE TREJO--SEGUNDINA IRIARTE', '2-jose-trejo-2', '0.8', '/imagen/foto_propiedad.jpg', 'activo', '359', 413, 14, NOW(), NOW());

SyntaxError: invalid syntax (312947826.py, line 2)

In [59]:
# seleccionar todas las porops activas
select = 'select * from propiedad where estado=\'activo\''
select_prop = dbQuery(select)

In [66]:
# convertir a df
df_props = pd.DataFrame(select_prop)
# convertir lista
list_props = list(df_props['codigo'])
# convertir a enteros
props_agrocittca = [int(i) for i in list_props]
# convertir a set
props_agrocittca = set(props_agrocittca)

In [67]:
len(props_agrocittca)

1615

In [68]:
# codigos de catastro
props_cat = set(list(data['cod_prop']))

In [69]:
len(props_cat)

1459

In [70]:
# codigos de catastro - codigos de agrocittca
props_select = props_cat.difference(props_agrocittca)

In [71]:
len(props_select)

0

In [48]:
for i in props_select:
    r = data[data['cod_prop']==i]
    prop = {
        'codigo' : str(r['cod_prop'].iloc[0]), 
        'nombre' : str(r['nom_prop'].iloc[0]), 
        'slug' : '', 
        'area' : str(r['area'].iloc[0]), 
        'foto' : '/imagen/foto_propiedad.jpg', 
        'estado' : 'activo', 
        'cuadrante' : str(r['cuadrante'].iloc[0]), 
        'usuario' : r['user_id'].iloc[0], 
        'periodo' : r['periodo_id'].iloc[0]
    }
    sql_insert = 'INSERT INTO propiedad (codigo, nombre, slug, superficie_total, foto, estado, cuadrante, usuario_id, periodo_id, created_at, updated_at) VALUES (\'{codigo}\', \'{nombre}\', \'{slug}\', \'{area}\', \'{foto}\', \'{estado}\', \'{cuadrante}\', {usuario}, {periodo}, NOW(), NOW())'.format(codigo=prop['codigo'], nombre=prop['nombre'], slug=prop['slug'], area=prop['area'], foto=prop['foto'], estado=prop['estado'], cuadrante=prop['cuadrante'], usuario=prop['usuario'], periodo=prop['periodo'])
    insert_row = dbQuery(sql_insert, 1)
    select_reg = 'select * from propiedad where codigo=\'{cod}\''.format(cod=prop['codigo'])
    select_reg = dbQuery(select_reg)
    dic = select_reg[0]
    slug_aux = abbreviate_para_slug(dic['nombre']) + '-' + str(dic['id'])
    update = 'update propiedad set slug=\'{sl}\' where id = {cod}'.format(sl=slug_aux, cod=dic['id'])
    update_row = dbQuery(update, 1)
    print('Registro de propiedad:', prop['nombre'])

Registro de propiedad: 5 - SAN FELIX--CATALA
Registro de propiedad: 6 - MONTE CLARO--MORA
Registro de propiedad: 8 - VILLA FRANCA--REMBERTO SUAREZ
Registro de propiedad: 9 - EL TAQUIAL--RICARDO JIMENEZ
Registro de propiedad: 10 - PUESTO MENDEZ--INSERVA DAMIANA S.R.L.
Registro de propiedad: 11 - EL CARMENCITO--ROLY LEON
Registro de propiedad: 12 - VALENTIN VALLEJOS--CARLOS VALLEJOS
Registro de propiedad: 13 - GUAYABOCHI--AGUILERA
Registro de propiedad: 14 - LOS CORRALES--ZUMARAN
Registro de propiedad: 15 - LA COLORADA
Registro de propiedad: 16 - LA VICTORIA--ROCA
Registro de propiedad: 21 - LOS MAGUESES--CIRBIAN
Registro de propiedad: 23 - CAÑUELAR
Registro de propiedad: 26 - MONACO--HUANCA
Registro de propiedad: 27 - EL CUQUITO
Registro de propiedad: 28 - LA PATRIA--A.MARIA FARREL
Registro de propiedad: 38 - AGUA DULCE--DIONICIO RICALDEZ
Registro de propiedad: 39 - AGUA DULCE--ELISA LEON
Registro de propiedad: 40 - SAN VICENTE--ERWIN BEJARANO
Registro de propiedad: 42 - EL BOLSON--JOSE

2023-08-30 15:15:00,541| ERROR   | Could not establish connection from local ('127.0.0.1', 51900) to remote ('127.0.0.1', 3306) side of the tunnel: 


Registro de propiedad: 370 - PORORO--PERALES
Registro de propiedad: 375 - SANTA ANITA--SANTIAGO GUILLERMO
Registro de propiedad: 376 - NARANJAL--SANTIAGO AGUILERA
Registro de propiedad: 377 - LA FINCA--RAUL VARGAS
Registro de propiedad: 378 - LA ALTURA--RAUL VARGAS
Registro de propiedad: 382 - NIVIA I--MONTAÑO NORAH
Registro de propiedad: 383 - NIVIA II--MONTAÑO NORAH
Registro de propiedad: 384 - AVAROA I--VICTOR CAHUASIRI
Registro de propiedad: 386 - AVAROA II--VICTOR CAHUASIRI
Registro de propiedad: 389 - SAN FELIX---FELICIANO ROJAS
Registro de propiedad: 395 - BARRIO LINDO-COPAIBO-LOS MANGALES--HUALLPA
Registro de propiedad: 396 - CURICHI III--PEDRO HUALLPA
Registro de propiedad: 398 - SAN MARTIN--JUAN C. RIOS
Registro de propiedad: 399 - LA CAÑADA--CASIANO CHOQUE
Registro de propiedad: 403 - ESMERALDA--MARCELO MORA
Registro de propiedad: 404 - SANTA MARIA--MORA
Registro de propiedad: 405 - AGUA ZARCA--MARCELO MORA
Registro de propiedad: 406 - EL CUADRO--MORA
Registro de propiedad: 

In [18]:
sql_insert

"INSERT INTO propiedad (codigo, nombre, slug, superficie_total, foto, estado, cuadrante, usuario_id, periodo_id, created_at, updated_at) VALUES ('1991', '1991 - SAN LORENZO II--ESCALERA JORGE', '', '33.94', '/imagen/foto_propiedad.jpg', 'activo', '0', 838, 14, NOW(), NOW())"

In [19]:
insert_row = dbQuery(sql_insert, 1)

In [20]:
select_reg = 'select * from propiedad where codigo=\'{cod}\''.format(cod=1991)
select_reg = dbQuery(select_reg)

In [21]:
dic = select_reg[0]
slug_aux = abbreviate_para_slug(dic['nombre']) + '-' + str(dic['id'])

In [22]:
dic

{'id': 355,
 'codigo': '1991',
 'nombre': '1991 - SAN LORENZO II--ESCALERA JORGE',
 'slug': '',
 'superficie_total': '33.94',
 'foto': '/imagen/foto_propiedad.jpg',
 'estado': 'activo',
 'enlace': None,
 'cuadrante': '0',
 'usuario_id': 838,
 'periodo_id': 14,
 'created_at': datetime.datetime(2023, 8, 30, 16, 4, 51),
 'updated_at': datetime.datetime(2023, 8, 30, 16, 4, 51)}

In [23]:
slug_aux

'1991-san-lorenzo-ii-escalera-jorge-355'

In [129]:
update propiedad set slug=slug_aux where id = 353

SyntaxError: invalid syntax (680471579.py, line 1)

In [28]:
update = 'update propiedad set slug=\'{sl}\' where id = {cod}'.format(sl=slug_aux, cod=dic['id'])

In [29]:
update

"update propiedad set slug='1991-san-lorenzo-ii-escalera-jorge-355' where id = 355"

In [30]:
update_row = dbQuery(update, 1)